In [52]:
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from typing import List

In [53]:
def generate_year_month_range(end_date: date = None, years_back: int = 6) -> List[str]:
    """
    Generate a list of year-month combinations in 'yyyymm' format,
    starting from the specified end date and going back a specified number of years.

    Args:
        end_date (date, optional): The end date to start from. Defaults to today's date.
        years_back (int, optional): Number of years to go back. Defaults to 6.

    Returns:
        List[str]: List of year-month combinations in 'yyyymm' format, sorted in descending order.

    Example:
        >>> generate_year_month_range()  # If today is 2024-11-14
        ['202411', '202410', '202409', ..., '201812']
    """
    # If no end date is provided, use today's date
    if end_date is None:
        end_date = date.today()

    # Calculate start date
    start_date = end_date - relativedelta(years=years_back)

    # Initialize result list
    date_list = []

    # Current date for iteration
    current_date = end_date

    # Generate dates until we reach start date
    while current_date >= start_date:
        # Format date as 'yyyymm'
        date_str = current_date.strftime('%Y%m')
        date_list.append(date_str)
        # Move to previous month
        current_date -= relativedelta(months=1)

    return date_list

# Example usage
if __name__ == "__main__":
    date_list = generate_year_month_range()
    print(f"Generated {len(date_list)} year-month combinations:")
    print(date_list[:12])  # Print first year as example

Generated 73 year-month combinations:
['202501', '202412', '202411', '202410', '202409', '202408', '202407', '202406', '202405', '202404', '202403', '202402']


In [54]:
import requests

In [55]:
# read in the metro_areas.json file as a list
import json
with open('data/metro_areas.json') as f:
    metro_areas = json.load(f)

metro_areas

['Abilene, TX',
 'Akron, OH',
 'Albany, OR',
 'Albany-Schenectady-Troy, NY',
 'Albuquerque, NM',
 'Allentown-Bethlehem-Easton, PA-NJ',
 'Altoona, PA',
 'Amarillo, TX',
 'Ames, IA',
 'Amherst Town-Northampton, MA',
 'Ann Arbor, MI',
 'Appleton, WI',
 'Asheville, NC',
 'Athens-Clarke County, GA',
 'Atlanta-Sandy Springs-Roswell, GA',
 'Atlantic City-Hammonton, NJ',
 'Auburn-Opelika, AL',
 'Baltimore-Columbia-Towson, MD',
 'Barnstable Town, MA',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Birmingham, AL',
 'Bloomington, IL',
 'Bloomington, IN',
 'Boise City, ID',
 'Boston-Cambridge-Newton, MA-NH',
 'Boulder, CO',
 'Bowling Green, KY',
 'Bremerton-Silverdale-Port Orchard, WA',
 'Bridgeport-Stamford-Danbury, CT',
 'Brownsville-Harlingen, TX',
 'Buffalo-Cheektowaga, NY',
 'Burlington, NC',
 'Burlington-South Burlington, VT',
 'Canton-Massillon, OH',
 'Cape Coral-Fort Myers, FL',
 'Cape Girardeau, MO-IL',
 'Carson City, NV',
 'Cedar Rapids, IA',
 'Chambersburg, PA',
 'Champaig

In [56]:
import pandas as pd
import requests
from datetime import datetime

def check_url_exists(url):
    """Check if a URL exists without downloading the full file"""
    try:
        response = requests.head(url)
        return response.status_code == 200
    except:
        return False

def process_2024_data(url):
    """Process data from 2024 onwards"""
    df = pd.read_excel(url, skiprows=7)
    
    # Clean columns
    df.drop(columns=['Metro /Micro Code', 'Unnamed: 10'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

def process_2022_2023_data(url):
    """Process data from 2022-2023"""
    df = pd.read_excel(url, skiprows=7)
    
    # Remove first row and clean columns
    df = df.iloc[1:]
    df.drop(columns=['Unnamed: 9'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

def process_pre_2022_data(url):
    """Process data from 2021 and earlier"""
    df = pd.read_excel(url, skiprows=7)
    
    # Remove first row and clean columns
    df = df.iloc[1:]
    df.drop(columns=['Monthly Coverage Percent*', 'Unnamed: 10'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

# Initialize an empty list to store all dataframes
all_dfs = []

# Process each date
for date_str in date_list:
    year = int(date_str[:4])
    
    try:
        # Determine URL format and processing function based on year
        if year >= 2024:
            url = f'https://www.census.gov/construction/bps/xls/cbsamonthly_{date_str}.xls'
            process_func = process_2024_data
        else:
            url = f'https://www.census.gov/construction/bps/xls/msamonthly_{date_str}.xls'
            if year >= 2022:
                process_func = process_2022_2023_data
            else:
                process_func = process_pre_2022_data
        
        # Check if URL exists
        if check_url_exists(url):
            try:
                df = process_func(url)
                all_dfs.append(df)
                print(f"Successfully processed data for {date_str}")
            except Exception as e:
                print(f"Error processing {date_str}: {str(e)}")
                continue
        else:
            print(f"No data available for {date_str}")
            continue
            
    except Exception as e:
        print(f"Error with {date_str}: {str(e)}")
        continue

# Concatenate all dataframes if we have any data
if all_dfs:
    homebuilding = pd.concat(all_dfs, ignore_index=True)
    
    # Convert date column to datetime
    homebuilding['date'] = pd.to_datetime(homebuilding['date'], format='%Y%m')
    
    # Sort by date and other relevant columns
    homebuilding = homebuilding.sort_values(['date', 'Name'], ascending=[False, True])
    
    print(f"\nFinal dataset contains {len(homebuilding)} rows from {len(all_dfs)} different months")
    print(f"Date range: {homebuilding['date'].min()} to {homebuilding['date'].max()}")
else:
    print("No data was successfully processed")
    homebuilding = pd.DataFrame()

No data available for 202501
No data available for 202412


/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/2982068359.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.1', '_ytd')


Successfully processed data for 202411


/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/2982068359.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.1', '_ytd')


Successfully processed data for 202410


...

Successfully processed data for 202001


/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/2982068359.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.1', '_ytd')


Successfully processed data for 201912


/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/2982068359.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.1', '_ytd')


Successfully processed data for 201911
No data available for 201910
No data available for 201909
No data available for 201908
No data available for 201907
No data available for 201906
No data available for 201905
No data available for 201904
No data available for 201903
No data available for 201902
No data available for 201901

Final dataset contains 28915 rows from 61 different months
Date range: 2019-11-01 00:00:00 to 2024-11-01 00:00:00


In [57]:
# homebuilding['date'] = pd.to_datetime(homebuilding['date'])
homebuilding.dtypes

CSA                                                  float64
CBSA                                                 float64
Name                                                  object
Total                                                float64
1 Unit                                               float64
2 Units                                              float64
3 and 4 Units                                        float64
5 Units or More                                      float64
Num of Structures With 5 Units or More               float64
Total_ytd                                            float64
1 Unit_ytd                                           float64
2 Units_ytd                                          float64
3 and 4 Units_ytd                                    float64
5 Units or More_ytd                                  float64
Num of Structures With 5 Units or More_ytd           float64
date                                          datetime64[ns]
dtype: object

In [58]:
homebuilding.columns

Index(['CSA', 'CBSA', 'Name', 'Total', '1 Unit', '2 Units', '3 and 4 Units',
       '5 Units or More', 'Num of Structures With 5 Units or More',
       'Total_ytd', '1 Unit_ytd', '2 Units_ytd', '3 and 4 Units_ytd',
       '5 Units or More_ytd', 'Num of Structures With 5 Units or More_ytd',
       'date'],
      dtype='object')

In [59]:
# create a 'multi_total' column that sums the columns for different types of multi-unit structures
homebuilding['multi_total'] = homebuilding['2 Units'] + homebuilding['3 and 4 Units'] + homebuilding['5 Units or More'] + homebuilding['Num of Structures With 5 Units or More']

# create a 'multi_total_ytd' column that sums the columns for different types of multi-unit structures
homebuilding['multi_total_ytd'] =  homebuilding['2 Units_ytd'] + homebuilding['3 and 4 Units_ytd'] + homebuilding['5 Units or More_ytd'] + homebuilding['Num of Structures With 5 Units or More_ytd']

In [60]:
homebuilding

,CSA,CBSA,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Total_ytd,1 Unit_ytd,2 Units_ytd,3 and 4 Units_ytd,5 Units or More_ytd,Num of Structures With 5 Units or More_ytd,date,multi_total,multi_total_ytd
0,999.0,10100.0,"Aberdeen, SD",7.0,3.0,4.0,0.0,0.0,0.0,87.0,71.0,12.0,4.0,0.0,0.0,2024-11-01,4.0,16.0
1,999.0,10140.0,"Aberdeen, WA",13.0,13.0,0.0,0.0,0.0,0.0,262.0,225.0,8.0,12.0,17.0,2.0,2024-11-01,0.0,39.0
2,101.0,10180.0,"Abilene, TX",21.0,17.0,4.0,0.0,0.0,0.0,442.0,346.0,96.0,0.0,0.0,0.0,2024-11-01,4.0,96.0
3,999.0,10220.0,"Ada, OK",0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2024-11-01,0.0,0.0
4,220.0,10300.0,"Adrian, MI",10.0,10.0,0.0,0.0,0.0,0.0,128.0,128.0,0.0,0.0,0.0,0.0,2024-11-01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28910,148.0,49340.0,"Worcester, MA-CT",18.0,16.0,2.0,0.0,0.0,0.0,219.0,172.0,6.0,0.0,41.0,5.0,2019-11-01,2.0,52.0
28911,999.0,49420.0,"Yakima, WA",52.0,40.0,6.0,0.0,6.0,1.0,823.0,368.0,62.0,23.0,370.0,21.0,2019-11-01,13.0,476.0
28912,276.0,49620.0,"York-Hanover, PA",11.0,7.0,0.0,4.0,0.0,0.0,155.0,96.0,6.0,11.0,42.0,3.0,2019-11-01,4.0,62.0
28913,566.0,49660.0,"Youngstown-Warren-Boardman, OH-PA",13.0,13.0,0.0,0.0,0.0,0.0,200.0,193.0,4.0,3.0,0.0,0.0,2019-11-01,0.0,7.0


In [61]:
# rename Name to name, Total to total, and Total_ytd to total_ytd
homebuilding.rename(
    columns={
        'Name': 'name',
        'Total': 'total',
        'Total_ytd': 'total_ytd'
    },
    inplace=True
)

homebuilding = homebuilding[['date', 'name', 'total', 'total_ytd', 'multi_total', 'multi_total_ytd']]

homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd
0,2024-11-01,"Aberdeen, SD",7.0,87.0,4.0,16.0
1,2024-11-01,"Aberdeen, WA",13.0,262.0,0.0,39.0
2,2024-11-01,"Abilene, TX",21.0,442.0,4.0,96.0
3,2024-11-01,"Ada, OK",0.0,2.0,0.0,0.0
4,2024-11-01,"Adrian, MI",10.0,128.0,0.0,0.0
...,...,...,...,...,...,...
28910,2019-11-01,"Worcester, MA-CT",18.0,219.0,2.0,52.0
28911,2019-11-01,"Yakima, WA",52.0,823.0,13.0,476.0
28912,2019-11-01,"York-Hanover, PA",11.0,155.0,4.0,62.0
28913,2019-11-01,"Youngstown-Warren-Boardman, OH-PA",13.0,200.0,0.0,7.0


In [62]:
# # unify the names of the metro areas for new york city
# # replace all instances of 'New York-Newark-Jersey City, NY-NJ-PA' with 'New York-Newark-Jersey City, NY-NJ' 
# homebuilding['name'] = homebuilding['name'].str.replace('New York-Newark-Jersey City, NY-NJ-PA', 'New York-Newark-Jersey City, NY-NJ')

In [63]:
# sort by date with earliest date first
homebuilding = homebuilding.sort_values('date')



homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd
28914,2019-11-01,"Yuma, AZ",80.0,1063.0,0.0,6.0
28663,2019-11-01,"Fayetteville-Springdale-Rogers, AR-MO",495.0,6303.0,199.0,2562.0
28664,2019-11-01,"Flagstaff, AZ",48.0,650.0,19.0,194.0
28665,2019-11-01,"Flint, MI",18.0,371.0,0.0,130.0
28666,2019-11-01,"Florence, SC",36.0,664.0,6.0,288.0
...,...,...,...,...,...,...
608,2024-11-01,"Oak Harbor, WA",23.0,383.0,0.0,112.0
607,2024-11-01,"Norwich-New London-Willimantic, CT",12.0,470.0,0.0,259.0
606,2024-11-01,"Norwalk, OH",2.0,37.0,0.0,0.0
604,2024-11-01,"North Port-Bradenton-Sarasota, FL",693.0,13863.0,101.0,3849.0


In [64]:
# Change the name column so 'Austin-Round Rock-Georgetown, TX' becomes 'Austin, TX'
# Delete every character between the first '-' and the ','
homebuilding['name'] = homebuilding['name'].str.replace(r'-.*?,', ',')
homebuilding

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/2817639881.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  homebuilding['name'] = homebuilding['name'].str.replace(r'-.*?,', ',')


,date,name,total,total_ytd,multi_total,multi_total_ytd
28914,2019-11-01,"Yuma, AZ",80.0,1063.0,0.0,6.0
28663,2019-11-01,"Fayetteville, AR-MO",495.0,6303.0,199.0,2562.0
28664,2019-11-01,"Flagstaff, AZ",48.0,650.0,19.0,194.0
28665,2019-11-01,"Flint, MI",18.0,371.0,0.0,130.0
28666,2019-11-01,"Florence, SC",36.0,664.0,6.0,288.0
...,...,...,...,...,...,...
608,2024-11-01,"Oak Harbor, WA",23.0,383.0,0.0,112.0
607,2024-11-01,"Norwich, CT",12.0,470.0,0.0,259.0
606,2024-11-01,"Norwalk, OH",2.0,37.0,0.0,0.0
604,2024-11-01,"North Port, FL",693.0,13863.0,101.0,3849.0


In [65]:
# change the name of metro areas to match the names in the metro_areas.json file
homebuilding['name'] = homebuilding['name'].str.replace(r'-.*', '')
homebuilding

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_26390/3585228184.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  homebuilding['name'] = homebuilding['name'].str.replace(r'-.*', '')


,date,name,total,total_ytd,multi_total,multi_total_ytd
28914,2019-11-01,"Yuma, AZ",80.0,1063.0,0.0,6.0
28663,2019-11-01,"Fayetteville, AR",495.0,6303.0,199.0,2562.0
28664,2019-11-01,"Flagstaff, AZ",48.0,650.0,19.0,194.0
28665,2019-11-01,"Flint, MI",18.0,371.0,0.0,130.0
28666,2019-11-01,"Florence, SC",36.0,664.0,6.0,288.0
...,...,...,...,...,...,...
608,2024-11-01,"Oak Harbor, WA",23.0,383.0,0.0,112.0
607,2024-11-01,"Norwich, CT",12.0,470.0,0.0,259.0
606,2024-11-01,"Norwalk, OH",2.0,37.0,0.0,0.0
604,2024-11-01,"North Port, FL",693.0,13863.0,101.0,3849.0


In [66]:
# Ensure the DataFrame is sorted by 'name' and 'date'
homebuilding = homebuilding.sort_values(by=['name', 'date'])

# Create a 12-month running total column
homebuilding['rt'] = (
    homebuilding.groupby('name')['total']
    .rolling(window=12, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

# Create a 12-month running total column for multi-unit structures
homebuilding['multi_rt'] = (
    homebuilding.groupby('name')['multi_total']
    .rolling(window=12, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt
9210,2024-01-01,"Aberdeen, SD",2.0,2.0,0.0,0.0,2.0,0.0
8289,2024-02-01,"Aberdeen, SD",2.0,4.0,0.0,0.0,4.0,0.0
7368,2024-03-01,"Aberdeen, SD",4.0,8.0,0.0,0.0,8.0,0.0
6447,2024-04-01,"Aberdeen, SD",15.0,23.0,0.0,0.0,23.0,0.0
5526,2024-05-01,"Aberdeen, SD",1.0,24.0,0.0,0.0,24.0,0.0
...,...,...,...,...,...,...,...,...
4604,2024-07-01,"Zanesville, OH",14.0,67.0,13.0,51.0,67.0,51.0
3683,2024-08-01,"Zanesville, OH",17.0,82.0,14.0,65.0,84.0,65.0
2762,2024-09-01,"Zanesville, OH",15.0,97.0,12.0,77.0,99.0,77.0
1841,2024-10-01,"Zanesville, OH",16.0,113.0,14.0,91.0,115.0,91.0


In [67]:
# show the df sorted by multi_total_12_month_running in the max date
homebuilding[homebuilding['date'] == homebuilding['date'].max()].sort_values('multi_rt', ascending=False)

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt
596,2024-11-01,"New York, NY",4216.0,52213.0,3448.0,41343.0,56155.0,44841.0
206,2024-11-01,"Dallas, TX",4322.0,66043.0,1454.0,23115.0,67954.0,23073.0
56,2024-11-01,"Austin, TX",3059.0,30193.0,2012.0,15127.0,33568.0,16662.0
653,2024-11-01,"Phoenix, AZ",3654.0,42476.0,1706.0,14506.0,45772.0,15881.0
489,2024-11-01,"Los Angeles, CA",2860.0,22911.0,2034.0,12645.0,26984.0,15404.0
...,...,...,...,...,...,...,...,...
149,2024-11-01,"Centralia, IL",0.0,2.0,0.0,0.0,2.0,0.0
606,2024-11-01,"Norwalk, OH",2.0,37.0,0.0,0.0,37.0,0.0
597,2024-11-01,"Newberry, SC",14.0,149.0,0.0,0.0,150.0,0.0
153,2024-11-01,"Charleston, WV",13.0,106.0,0.0,0.0,142.0,0.0


In [68]:
# population

# read in the population data
metros = pd.read_csv('data/zori_metro_long_clean.csv')

# change 'date' to datetime
metros['date'] = pd.to_datetime(metros['date'])

metros

,name,date,zori,lat,lng,pop_2023
0,"New York, NY",2015-02-01,2255.133793,40.6943,-73.9249,19498249.0
1,"Los Angeles, CA",2015-02-01,1814.136486,34.1141,-118.4068,12799100.0
2,"Chicago, IL",2015-02-01,1356.915213,41.8375,-87.6866,9262825.0
3,"Dallas, TX",2015-02-01,1090.919667,32.7935,-96.7667,8100037.0
4,"Houston, TX",2015-02-01,1229.410303,29.7860,-95.3885,7510253.0
...,...,...,...,...,...,...
5945,"Salt Lake City, UT",2024-12-01,1650.452457,40.7776,-111.9311,1267864.0
5946,"Hartford, CT",2024-12-01,1874.958152,41.7661,-72.6834,1151543.0
5947,"Buffalo, NY",2024-12-01,1344.539267,42.9018,-78.8487,1155604.0
5948,"Birmingham, AL",2024-12-01,1365.603146,33.5279,-86.7971,1184290.0


In [69]:
# merge the population data with the homebuilding data
# merge on 'name' and 'date'
# only include rows where both 'name' and 'date' are in both dataframes
homebuilding_zori = pd.merge(
    homebuilding,
    metros,
    on=['name', 'date'],
    how='inner'
)

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,-84.4220,6307261.0
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,-84.4220,6307261.0
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,-84.4220,6307261.0
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,-84.4220,6307261.0
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,-84.4220,6307261.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,-77.0163,6304975.0
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,-77.0163,6304975.0
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,-77.0163,6304975.0
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,-77.0163,6304975.0


In [70]:
homebuilding_zori.columns

Index(['date', 'name', 'total', 'total_ytd', 'multi_total', 'multi_total_ytd',
       'rt', 'multi_rt', 'zori', 'lat', 'lng', 'pop_2023'],
      dtype='object')

In [71]:
# create 'total_per_capita' and 'total_ytd_per_capita' columns that divide 'total' and 'total_ytd' by 'pop_2023'
homebuilding_zori['total_pc'] = homebuilding_zori['total'] / homebuilding_zori['pop_2023']
homebuilding_zori['total_ytd_pc'] = homebuilding_zori['total_ytd'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_total_pc'] = homebuilding_zori['multi_total'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_total_ytd_pc'] = homebuilding_zori['multi_total_ytd'] / homebuilding_zori['pop_2023']
homebuilding_zori['rt_pc'] = homebuilding_zori['rt'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_rt_pc'] = homebuilding_zori['multi_rt'] / homebuilding_zori['pop_2023']


# change per capita columns to per 1000
for col in homebuilding_zori.columns:
    if 'pc' in col:
        homebuilding_zori[col] = homebuilding_zori[col] * 1000

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,-77.0163,6304975.0,0.314514,2.154965,0.157336,1.013168,3.580030,1.852188
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318


In [72]:
# create 'zori_yoy' column that calculates the year-over-year change in 'zori' for each metro area
homebuilding_zori['zori_yoy'] = homebuilding_zori.groupby('name')['zori'].pct_change(12)
homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,-77.0163,6304975.0,0.314514,2.154965,0.157336,1.013168,3.580030,1.852188,0.050730
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050124
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.049168
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047615


In [73]:
# sort by multi_rt_pc in the max date and filter to this date
homebuilding_zori[homebuilding_zori['date'] == homebuilding_zori['date'].max()].sort_values('rt_pc', ascending=False)

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy
121,2024-11-01,"Austin, TX",3059.0,30193.0,2012.0,15127.0,33568.0,16662.0,1708.494615,30.3005,-97.7522,2473275.0,1.236822,12.207700,0.813496,6.116182,13.572288,6.736817,-0.032811
2256,2024-11-01,"Raleigh, NC",879.0,18087.0,52.0,5584.0,20304.0,7130.0,1745.296204,35.8324,-78.6429,1509231.0,0.582416,11.984249,0.034455,3.699897,13.453209,4.724260,0.003932
1707,2024-11-01,"Nashville, TN",1633.0,18699.0,716.0,5262.0,19830.0,5479.0,1866.113550,36.1715,-86.7842,2102573.0,0.776667,8.893389,0.340535,2.502648,9.431302,2.605855,0.015025
426,2024-11-01,"Charlotte, NC",1786.0,23814.0,491.0,6420.0,25936.0,7149.0,1775.160145,35.2083,-80.8303,2805115.0,0.636694,8.489492,0.175037,2.288676,9.245967,2.548559,0.013154
1158,2024-11-01,"Jacksonville, FL",785.0,13943.0,4.0,2106.0,15766.0,2833.0,1709.775642,30.3322,-81.6749,1713240.0,0.458196,8.138381,0.002335,1.229250,9.202447,1.653592,0.015999
2012,2024-11-01,"Phoenix, AZ",3654.0,42476.0,1706.0,14506.0,45772.0,15881.0,1825.953865,33.5722,-112.0892,5070110.0,0.720694,8.377728,0.336482,2.861082,9.027812,3.132279,0.008782
1036,2024-11-01,"Houston, TX",5099.0,61141.0,1501.0,12408.0,64612.0,12792.0,1696.645183,29.7860,-95.3885,7510253.0,0.678939,8.141004,0.199860,1.652141,8.603172,1.703272,0.027611
731,2024-11-01,"Dallas, TX",4322.0,66043.0,1454.0,23115.0,67954.0,23073.0,1743.874174,32.7935,-96.7667,8100037.0,0.533578,8.153420,0.179505,2.853691,8.389344,2.848506,0.002903
1890,2024-11-01,"Orlando, FL",1388.0,21421.0,142.0,7551.0,22901.0,7656.0,2010.439552,28.4773,-81.3370,2817933.0,0.492560,7.601671,0.050392,2.679624,8.126879,2.716885,0.013152
2927,2024-11-01,"Tampa, FL",921.0,20057.0,18.0,8015.0,22665.0,8680.0,2049.558445,27.9945,-82.4447,3342963.0,0.275504,5.999767,0.005384,2.397574,6.779914,2.596499,0.018009


In [74]:

with open('data/top_metros.json') as f:
    top_metros = json.load(f)

# filter to only the top metros
homebuilding_zori = homebuilding_zori[homebuilding_zori['name'].isin(top_metros)]

In [75]:
homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,-77.0163,6304975.0,0.314514,2.154965,0.157336,1.013168,3.580030,1.852188,0.050730
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050124
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.049168
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047615


In [76]:
# Create a 'state' column by extracting the state abbreviation from the 'name' column
homebuilding_zori['state'] = homebuilding_zori['name'].str.split(', ').str[-1]

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy,state
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN,GA
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN,GA
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN,GA
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN,GA
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN,GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,-77.0163,6304975.0,0.314514,2.154965,0.157336,1.013168,3.580030,1.852188,0.050730,DC
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050124,DC
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.049168,DC
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047615,DC


In [77]:
regions = pd.read_csv('data/regions.csv')

# merge
homebuilding_zori = pd.merge(
    homebuilding_zori,
    regions,
    on='state',
    how='left'
)

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,...,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy,state,region
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1376.046670,33.7628,...,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN,GA,Southeast
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1373.491783,33.7628,...,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN,GA,Southeast
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1373.374324,33.7628,...,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN,GA,Southeast
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1378.194661,33.7628,...,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN,GA,Southeast
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1386.720356,33.7628,...,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN,GA,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,2024-07-01,"Washington, DC",1983.0,13587.0,992.0,6388.0,22572.0,11678.0,2365.897539,38.9047,...,6304975.0,0.314514,2.154965,0.157336,1.013168,3.580030,1.852188,0.050730,DC,Northest
3046,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2381.806738,38.9047,...,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050124,DC,Northest
3047,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2390.051806,38.9047,...,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.049168,DC,Northest
3048,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2389.848641,38.9047,...,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047615,DC,Northest


In [78]:
# save the final dataset to a csv file
homebuilding_zori.to_csv('data/homebuilding_zori.csv', index=False)

In [44]:
# make a list of the names of every state and District of Columbia
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [51]:
nov24 = "https://www.census.gov/construction/bps/xls/statemonthly_202411.xls"

df = pd.read_excel(nov24, skiprows=5)

# rename columns
df.rename(
    columns={
        'Unnamed: 0': 'Name'
    },
    inplace=True
)

# drop null names
df = df.dropna(subset=['Name'])

# drop rows where 'Name includes 'Region', 'Division', or 'United States'
df = df[~df['Name'].str.contains('Region|Division|United States')]

# drop columns that have 'Unnamed' in the name
df = df.loc[:, ~df.columns.str.contains('^Unnamed|Year to Date')]

# add date column
df['date'] = '202411'

# make it datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m')

df

,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,date
7,Connecticut,564.0,146.0,2.0,9.0,407.0,15.0,2024-11-01
8,Maine,387.0,317.0,18.0,15.0,37.0,6.0,2024-11-01
9,Massachusetts,1006.0,355.0,24.0,6.0,621.0,36.0,2024-11-01
10,New Hampshire,336.0,224.0,10.0,19.0,83.0,7.0,2024-11-01
11,Rhode Island,90.0,63.0,24.0,3.0,0.0,0.0,2024-11-01
12,Vermont,132.0,112.0,2.0,6.0,12.0,2.0,2024-11-01
15,New Jersey,3179.0,1110.0,60.0,40.0,1969.0,84.0,2024-11-01
16,New York,3492.0,775.0,66.0,40.0,2611.0,86.0,2024-11-01
17,Pennsylvania,1800.0,1208.0,42.0,44.0,506.0,27.0,2024-11-01
21,Illinois,1819.0,779.0,58.0,87.0,895.0,34.0,2024-11-01
